In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000606585s, CPU 0.000579s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0074251s, CPU 0.007409s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction)

Problem("body1-2D", :PlaneHeatConduction, 2, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
#T0 = FEM.initialTemperature(problem, "body", T=0)
T0 = scalarField(problem, "body", 0)

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [6]:
#FEM.initialTemperature!("left", T0, T=0)
#T0

In [7]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

ScalarField
[0.0; -0.062499999999999924; … ; 0.0; 0.0;;]

In [8]:
#FEM.applyBoundaryConditions!(K, C, q, [supp])

In [9]:
λmax = FEM.largestEigenValue(K, C, [supp])

28634.16628528278

In [10]:
ϑ = 0.33
T = FEM.FDM(K, C, q, [supp], T0, 20, 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 -0.0060297789879445104 … 5.641254670169727 -10.1564878089319; … ; 0.0 -1.6743417968164548e-6 … 0.021679854915270885 -0.05736439147570293; 0.0 -0.00038000429880581085 … -0.21748580922588315 0.8924961347251381]

In [13]:
ϑ = 0.66
T = FEM.FDM(K, C, q, [supp], T0, 15, 0.01, ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 -0.009997131200716585 … -0.021951429446714445 -0.02202612204493276; … ; 0.0 -0.0006955028498644954 … -0.005673850726920832 -0.00570353264939802; 0.0 -0.004299019693876791 … -0.016393746191050843 -0.016462740381772187]

In [14]:
FEM.showDoFResults(T, :T, visible=true)

1

In [15]:
q1 = FEM.solveHeatFlux(T)

VectorField
[[0.0 0.8737295094799347 … 0.9950319310459317 0.9964232844631155; 0.0 -0.004516731957424705 … -4.5151742416177365e-5 -3.3471992804956496e-5; … ; 0.0 0.8737295094799347 … 0.9950319310459317 0.9964232844631155; 0.0 -0.004516731957424705 … -4.5151742416177365e-5 -3.3471992804956496e-5], [0.0 0.8752832765651972 … 0.9950336078296282 0.9964242720440222; 0.0 -0.006883298036587625 … -4.7705676950032805e-5 -3.497619217626702e-5; … ; 0.0 0.8752832765651972 … 0.9950336078296282 0.9964242720440222; 0.0 -0.006883298036587625 … -4.7705676950032805e-5 -3.497619217626702e-5], [0.0 0.85191190193785 … 0.9948052809402799 0.9962591186700946; 0.0 0.02185624188389719 … 0.00019636914843042426 0.00013998047680150068; … ; 0.0 0.85191190193785 … 0.9948052809402799 0.9962591186700946; 0.0 0.02185624188389719 … 0.00019636914843042426 0.00013998047680150068], [0.0 0.12874648956120277 … 0.9817238415478612 0.9867701484671473; 0.0 0.003886736606651777 … 0.00014282531649789307 0.00010343592529805934; … ; 0

In [16]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [17]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

VectorField
[0.0 0.10867379353969966 … 0.9809659091685992 0.9862212190272401; 0.0 -0.00044790216297533984 … -1.7803541702684708e-5 -1.289491349776642e-5; … ; 0.0 0.6669871496499239 … 0.992618796192568 0.9946654638767285; 0.0 0.0011124294887966946 … 7.1733908007382e-6 5.034886550077389e-6]

In [18]:
FEM.showDoFResults(q2, :qvec)

3

In [19]:
q3 = FEM.elementsToNodes(q1)

VectorField
[0.0 0.10867379353969966 … 0.9809659091685992 0.9862212190272401; 0.0 -0.00044790216297533984 … -1.7803541702684708e-5 -1.289491349776642e-5; … ; 0.0 0.6669871496499239 … 0.992618796192568 0.9946654638767285; 0.0 0.0011124294887966946 … 7.1733908007382e-6 5.034886550077389e-6]

In [20]:
FEM.showDoFResults(q3, :qvec)

4

In [21]:
err1 = FEM.fieldError(q1)

VectorField
[0.0 0.0004479021629753485 … 1.7803541702721137e-5 1.2894913497818461e-5; 0.0 0.00044790216297533984 … 1.7803541702684708e-5 1.289491349776642e-5; … ; 0.0 0.11320332214012804 … 0.0014187127541214993 0.0010355757458748516; 0.0 0.04859962542026274 … 0.0006229325653385764 0.0004543881746336643]

In [22]:
FEM.showDoFResults(err1, :vector)

5

In [23]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [24]:
gmsh.finalize()